**Pronostico de la evolución de casos activos de SARS-CoV-2 en Colombia**

El nuevo Coronavirus (COVID-19) ha sido catalogado por la Organización Mundial de la Salud como una emergencia en salud pública de importancia internacional (ESPII). Se han identificado casos en todos los continentes y, el 6 de marzo se confirmó el primer caso en Colombia.De acuerdo con la información aportada por el Instituto Nacional de Salud, para el mes de agosto ya se ha confirmado alrededor de 530000 casos con una recuperación del alrededor del 67% de la población y una tasa de mortalidad del 3.2%.

Con el aumento de casos activos en Colombia, se pretende genenerar una herramienta de reporte que permita visualizar predicciones en el corto y mediano plazo del total de casos confirmados, nuevos casos, casos activos, recuperados y muertes para las cinco principales ciudades de Colombia, utilizando técnicas estadísticas, de inteligencia artificial o modelos híbridos.




In [121]:
pip install sodapy

### Importar librerias 

In [122]:
from sodapy import Socrata
import numpy as np 
import re
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import seaborn as sns
import pandas as pd 
import random
import math
import time
import datetime as dt
from datetime import timedelta
import operator 
plt.style.use('fivethirtyeight')
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
#!pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
import statsmodels.api as sm
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
!pip install pyramid-arima
from pyramid.arima import auto_arima
std=StandardScaler()
#pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [123]:
covid_19  = Socrata("www.datos.gov.co", None)
results = covid_19.get_all("gt2j-8ykr", limit=800000)

### Llama el servicio de Datos.gov.co para consultar los datos con formato .json

In [124]:
results_df = pd.DataFrame.from_records(results)
results_df.describe()

,id_de_caso,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,nombre_grupo_etnico,fecha_de_muerte
count,590520,590520,590520,590520,590520,590520,590520,590520,590520,590520,961,590520,586677,429625,590520,429620,590520,962,280281,8139,19889
unique,590520,177,1017,944,37,6,110,4,5,6,48,599,173,167,173,2,33,43,4,88,161
top,472390,2020-07-27T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,30,M,En estudio,Leve,ESPAÑA,Asintomático,2020-08-11T00:00:00.000,2020-08-23T00:00:00.000,2020-08-19T00:00:00.000,Tiempo,11,724,Otro,ZENÚ,2020-07-30T00:00:00.000
freq,1,12103,204065,204065,204065,429620,15838,303973,560201,477190,265,69460,17415,14199,13056,340959,204065,265,253968,1839,365


### Convertir el conjunto de datos de .json a Dataframe de Pandas

In [125]:
results_df.tail(5)

,id_de_caso,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,nombre_grupo_etnico,fecha_de_muerte
590515,590556,2020-08-13T00:00:00.000,05001,Medellín,Antioquia,Casa,18,M,En estudio,Leve,NaN,2020-08-09T02:17:14.254,2020-08-24T00:00:00.000,NaN,2020-08-28T00:00:00.000,NaN,5,NaN,NaN,NaN,NaN
590516,590557,2020-08-01T00:00:00.000,05001,Medellín,Antioquia,Casa,20,M,En estudio,Leve,NaN,2020-07-28T02:17:14.254,2020-08-12T00:00:00.000,NaN,2020-08-28T00:00:00.000,NaN,5,NaN,NaN,NaN,NaN
590517,590558,2020-07-30T00:00:00.000,05001,Medellín,Antioquia,Casa,17,F,En estudio,Leve,NaN,2020-07-26T02:17:14.254,2020-08-10T00:00:00.000,NaN,2020-08-28T00:00:00.000,NaN,5,NaN,NaN,NaN,NaN
590518,590559,2020-08-14T00:00:00.000,05001,Medellín,Antioquia,Casa,18,F,En estudio,Leve,NaN,2020-08-10T02:17:14.254,2020-08-25T00:00:00.000,NaN,2020-08-28T00:00:00.000,NaN,5,NaN,NaN,NaN,NaN
590519,590560,2020-08-02T00:00:00.000,05001,Medellín,Antioquia,Casa,20,M,En estudio,Leve,NaN,2020-07-29T02:17:14.254,2020-08-13T00:00:00.000,NaN,2020-08-28T00:00:00.000,NaN,5,NaN,NaN,NaN,NaN


# Análisis exploratorio de los datos


In [126]:
results_df.count()

id_de_caso               590520
fecha_de_notificaci_n    590520
c_digo_divipola          590520
ciudad_de_ubicaci_n      590520
departamento             590520
atenci_n                 590520
edad                     590520
sexo                     590520
tipo                     590520
estado                   590520
pa_s_de_procedencia         961
fis                      590520
fecha_diagnostico        586677
fecha_recuperado         429625
fecha_reporte_web        590520
tipo_recuperaci_n        429620
codigo_departamento      590520
codigo_pais                 962
pertenencia_etnica       280281
nombre_grupo_etnico        8139
fecha_de_muerte           19889
dtype: int64

In [127]:
results_df.shape

(590520, 21)

In [128]:
results_df.isnull().sum()

id_de_caso                    0
fecha_de_notificaci_n         0
c_digo_divipola               0
ciudad_de_ubicaci_n           0
departamento                  0
atenci_n                      0
edad                          0
sexo                          0
tipo                          0
estado                        0
pa_s_de_procedencia      589559
fis                           0
fecha_diagnostico          3843
fecha_recuperado         160895
fecha_reporte_web             0
tipo_recuperaci_n        160900
codigo_departamento           0
codigo_pais              589558
pertenencia_etnica       310239
nombre_grupo_etnico      582381
fecha_de_muerte          570631
dtype: int64

In [129]:
results_df.head()

,id_de_caso,fecha_de_notificaci_n,c_digo_divipola,ciudad_de_ubicaci_n,departamento,atenci_n,edad,sexo,tipo,estado,pa_s_de_procedencia,fis,fecha_diagnostico,fecha_recuperado,fecha_reporte_web,tipo_recuperaci_n,codigo_departamento,codigo_pais,pertenencia_etnica,nombre_grupo_etnico,fecha_de_muerte
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,ITALIA,2020-02-27T00:00:00.000,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,PCR,11,380,Otro,NaN,NaN
1,2,2020-03-06T00:00:00.000,76111,Guadalajara de Buga,Valle del Cauca,Recuperado,34,M,Importado,Leve,ESPAÑA,2020-03-04T00:00:00.000,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,PCR,76,724,Otro,NaN,NaN
2,3,2020-03-07T00:00:00.000,05001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,ESPAÑA,2020-02-29T00:00:00.000,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000,PCR,5,724,Otro,NaN,NaN
3,4,2020-03-09T00:00:00.000,05001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,NaN,2020-03-06T00:00:00.000,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN,NaN
4,5,2020-03-09T00:00:00.000,05001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,NaN,2020-03-08T00:00:00.000,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,PCR,5,NaN,Otro,NaN,NaN


In [130]:
results_df.dtypes

id_de_caso               object
fecha_de_notificaci_n    object
c_digo_divipola          object
ciudad_de_ubicaci_n      object
departamento             object
atenci_n                 object
edad                     object
sexo                     object
tipo                     object
estado                   object
pa_s_de_procedencia      object
fis                      object
fecha_diagnostico        object
fecha_recuperado         object
fecha_reporte_web        object
tipo_recuperaci_n        object
codigo_departamento      object
codigo_pais              object
pertenencia_etnica       object
nombre_grupo_etnico      object
fecha_de_muerte          object
dtype: object

#Limpieza, transformación y filtrado del conjunto de datos 

In [131]:
df = (results_df[results_df["estado"] != "N/A"])[["id_de_caso", "fecha_de_notificaci_n", "ciudad_de_ubicaci_n", "atenci_n", "edad", "sexo", "tipo", "estado", "fis", "fecha_diagnostico", "fecha_recuperado", "fecha_reporte_web", "tipo_recuperaci_n", "fecha_de_muerte"]]

In [132]:
df.columns = ["ID_de_caso", "Fecha_de_notificación", "Ciudad_de_ubicación", "atención", "Edad", "Sexo", "Tipo", "Estado", "FIS", "Fecha_diagnostico", "Fecha_recuperado", "fecha_reporte_web", "Tipo_recuperación", "Fecha_de_muerte"]

In [133]:
df = df[df.Ciudad_de_ubicación.isin(["Bogotá D.C.", "Medellín", "Barranquilla", "Cali", "Cartagena de Indias"])]

In [134]:
df.shape

(340460, 14)

In [135]:
df.isnull().sum()

ID_de_caso                    0
Fecha_de_notificación         0
Ciudad_de_ubicación           0
atención                      0
Edad                          0
Sexo                          0
Tipo                          0
Estado                        0
FIS                           0
Fecha_diagnostico          2032
Fecha_recuperado          89126
fecha_reporte_web             0
Tipo_recuperación         89126
Fecha_de_muerte          330923
dtype: int64

### Cambio del formato de la fecha

In [136]:
new_df = df[["fecha_reporte_web", "atención", "Ciudad_de_ubicación"]]

In [137]:
dateRegex = re.compile(r"[a-zA-Z]\d\d\:\d\d\:\d\d\.\d\d\d")

In [138]:
new_df = new_df.replace(dateRegex, "",regex=True, inplace=False)
new_df['fecha_reporte_web'] = pd.to_datetime(new_df['fecha_reporte_web'])
new_df.head(5)

,fecha_reporte_web,atención,Ciudad_de_ubicación
0,2020-03-06,Recuperado,Bogotá D.C.
2,2020-03-09,Recuperado,Medellín
3,2020-03-11,Recuperado,Medellín
4,2020-03-11,Recuperado,Medellín
6,2020-03-11,Recuperado,Cartagena de Indias


### Dataframe para procesamiento

In [140]:
new_df["atención"] = new_df["atención"].replace(to_replace=["Casa", "Hospital", "Hospital UCI", "N/A"], value="ACTIVOS")
new_df["atención"] = new_df["atención"].replace(to_replace=["Recuperado"], value="RECUPERADOS")
new_df["atención"] = new_df["atención"].replace(to_replace=["Fallecido"], value="MUERTES")

In [141]:
confirmados = new_df
recuperados = new_df[new_df["atención"] == "RECUPERADOS"]
activos = new_df[new_df["atención"] == "ACTIVOS"]
muertos = new_df[new_df["atención"] == "MUERTES"]

In [142]:
new_df["Confirmados"]=1
new_df["Recuperados"]= np.where(new_df["atención"] == "RECUPERADOS", 1, 0)
new_df["Muertos"]= np.where(new_df["atención"] == "MUERTES", 1, 0)
new_df["Activos"]= np.where(new_df["atención"] == "ACTIVOS", 1, 0)
new_df.head()

,fecha_reporte_web,atención,Ciudad_de_ubicación,Confirmados,Recuperados,Muertos,Activos
0,2020-03-06,RECUPERADOS,Bogotá D.C.,1,1,0,0
2,2020-03-09,RECUPERADOS,Medellín,1,1,0,0
3,2020-03-11,RECUPERADOS,Medellín,1,1,0,0
4,2020-03-11,RECUPERADOS,Medellín,1,1,0,0
6,2020-03-11,RECUPERADOS,Cartagena de Indias,1,1,0,0


In [ ]:
new_df["fecha_reporte_web"]=pd.to_datetime(new_df["fecha_reporte_web"])

In [157]:
df_agrupado= new_df.groupby(["Ciudad_de_ubicación","fecha_reporte_web"]).agg({"Confirmados":'sum',"Recuperados":'sum',"Activos":'sum',"Muertos":'sum'})
df_agrupado.head()

Confirmados  ...  Muertos
Ciudad_de_ubicación fecha_reporte_web               ...         
Barranquilla        2020-03-16                   2  ...        0
                    2020-03-19                   3  ...        0
                    2020-03-21                   2  ...        0
                    2020-03-24                   2  ...        0
                    2020-03-25                   3  ...        0

[5 rows x 4 columns]

In [159]:
df_agrupado2= new_df.groupby(["fecha_reporte_web"]).agg({"Confirmados":'sum',"Recuperados":'sum',"Activos":'sum',"Muertos":'sum'})
df_agrupado2.head()

,Confirmados,Recuperados,Activos,Muertos
fecha_reporte_web,,,,
2020-03-06,1,1,0,0
2020-03-09,1,1,0,0
2020-03-11,5,5,0,0
2020-03-12,2,2,0,0
2020-03-13,2,2,0,0


In [160]:
df_agrupado2.shape

(173, 4)

In [161]:
df_agrupado2["Confirmados_acum"] =df_agrupado2["Confirmados"].cumsum()
df_agrupado2["Recuperados_acum"] =df_agrupado2["Recuperados"].cumsum()
df_agrupado2["Activos_acum"] =df_agrupado2["Activos"].cumsum()
df_agrupado2["Muertos_acum"] =df_agrupado2["Muertos"].cumsum()
df_agrupado2.head()

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum
fecha_reporte_web,,,,,,,,
2020-03-06,1,1,0,0,1,1,0,0
2020-03-09,1,1,0,0,2,2,0,0
2020-03-11,5,5,0,0,7,7,0,0
2020-03-12,2,2,0,0,9,9,0,0
2020-03-13,2,2,0,0,11,11,0,0


In [162]:
df_agrupado2.tail()

,Confirmados,Recuperados,Activos,Muertos,Confirmados_acum,Recuperados_acum,Activos_acum,Muertos_acum
fecha_reporte_web,,,,,,,,
2020-08-24,5714,314,5365,35,322680,250755,62528,9397
2020-08-25,4094,172,3887,35,326774,250927,66415,9432
2020-08-26,5985,242,5706,37,332759,251169,72121,9469
2020-08-27,4008,165,3831,12,336767,251334,75952,9481
2020-08-28,3693,0,3693,0,340460,251334,79645,9481


In [164]:
df_agrupado2["Días"]=df_agrupado2.index-df_agrupado2.index.min()
df_agrupado2["Días"]=df_agrupado2["Días"].dt.days

### Resumen general de los datos

In [168]:
print("Informacion General")

print("Número total de casos confirmados en las 5 principales ciudades de Colombia: ",df_agrupado2["Confirmados_acum"].iloc[-1])
print("Número total de casos recuperados en las 5 principales ciudades de Colombia: ",df_agrupado2["Recuperados_acum"].iloc[-1])
print("Número total de muertes en las 5 principales ciudades de Colombia: ",df_agrupado2["Muertos_acum"].iloc[-1])
print("Número total de casos Activos en las 5 principales ciudades de Colombia: ",df_agrupado2["Activos_acum"].iloc[-1])

print("Número aproximado de casos confirmados por día: ",np.round(df_agrupado2["Confirmados_acum"].iloc[-1]/df_agrupado2.shape[0]))
print("Número aproximado de casos recuperados por día: ",np.round(df_agrupado2["Recuperados_acum"].iloc[-1]/df_agrupado2.shape[0]))
print("Número aproximado de muertes por día: ",np.round(df_agrupado2["Muertos_acum"].iloc[-1]/df_agrupado2.shape[0]))

print("Nuevos casos confirmados: ",df_agrupado2["Confirmados_acum"].iloc[-1]-df_agrupado2["Confirmados_acum"].iloc[-2])


Informacion General
Número total de casos confirmados en las 5 principales ciudades de Colombia:  340460
Número total de casos recuperados en las 5 principales ciudades de Colombia:  251334
Número total de muertes en las 5 principales ciudades de Colombia:  9481
Número total de casos Activos en las 5 principales ciudades de Colombia:  79645
Número aproximado de casos confirmados por día:  1968.0
Número aproximado de casos recuperados por día:  1453.0
Número aproximado de muertes por día:  55.0
Nuevos casos confirmados:  3693


In [169]:
df_agrupado2["Semana"]=df_agrupado2.index.weekofyear

semana_num=[]
semana_confirmados=[]
semana_recuperados=[]
semana_muertos=[]
w=1
for i in list(df_agrupado2["Semana"].unique()):
    semana_confirmados.append(df_agrupado2[df_agrupado2["Semana"]==i]["Confirmados"].iloc[-1])
    semana_recuperados.append(df_agrupado2[df_agrupado2["Semana"]==i]["Recuperados"].iloc[-1])
    semana_muertos.append(df_agrupado2[df_agrupado2["Semana"]==i]["Muertos"].iloc[-1])
    semana_num.append(w)
    w=w+1

fig=go.Figure()
fig.add_trace(go.Scatter(x=semana_num, y=semana_confirmados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos confirmados'))
fig.add_trace(go.Scatter(x=semana_num, y=semana_recuperados,
                    mode='lines+markers',
                    name='Crecimiento semanal casos recuperados'))
fig.add_trace(go.Scatter(x=semana_num, y=semana_muertos,
                    mode='lines+markers',
                    name='Crecimiento semanal muertos'))
fig.update_layout(title="Crecimiento semanal por tipo de casos en las 5 principales ciudades de Colombia",
                 xaxis_title="Semana",yaxis_title="Número de casos",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()


In [172]:
#tasa de mortalidad y recuperación

df_agrupado2["Tasa de mortalidad"]=(df_agrupado2["Muertos_acum"]/df_agrupado2["Confirmados_acum"])*100
df_agrupado2["Tasa de recuperación"]=(df_agrupado2["Recuperados_acum"]/df_agrupado2["Confirmados_acum"])*100

print("Tasa de mortalidad promedio",df_agrupado2["Tasa de mortalidad"].mean())
print("Tasa de recuperación promedio",df_agrupado2["Tasa de recuperación"].mean())



Tasa de mortalidad promedio 4.2112888123327386
Tasa de recuperación promedio 93.15513469161459


In [173]:
fig = make_subplots(rows=2, cols=1,
                   subplot_titles=("Tasa de recuperación", "Tasa de mortalidad"))
fig.add_trace(
    go.Scatter(x=df_agrupado2.index, y=(df_agrupado2["Recuperados_acum"]/df_agrupado2["Confirmados_acum"])*100,name="Tasa de recuperación"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=df_agrupado2.index, y=(df_agrupado2["Muertos_acum"]/df_agrupado2["Confirmados_acum"])*100,name="Tasa de mortalidad"),
    row=2, col=1
)
fig.update_layout(height=1000,legend=dict(x=-0.1,y=1.2,traceorder="normal"))
fig.update_xaxes(title_text="Fecha", row=1, col=1)
fig.update_yaxes(title_text="tasa de recuperación", row=1, col=1)
fig.update_xaxes(title_text="Fecha", row=1, col=2)
fig.update_yaxes(title_text="Tasa de mortalidad", row=1, col=2)
fig.show()

In [176]:
print("Factor de crecimiento promedio casos confirmados: ",(df_agrupado2["Confirmados"]/df_agrupado2["Confirmados"].shift()).mean())

print("Factor de crecimiento promedio casos recuperados: ",(df_agrupado2["Recuperados"]/df_agrupado2["Recuperados"].shift()).mean())

print("Factor de crecimiento promedio muertes: ",(df_agrupado2["Muertos"]/df_agrupado2["Muertos"].shift()).mean())


fig=go.Figure()
fig.add_trace(go.Scatter(x=df_agrupado2.index, y=df_agrupado2["Confirmados"]/df_agrupado2["Confirmados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos confirmados'))
fig.add_trace(go.Scatter(x=df_agrupado2.index, y=df_agrupado2["Recuperados"]/df_agrupado2["Recuperados"].shift(),
                    mode='lines',
                    name='Factor de crecimiento casos recuperados'))
fig.add_trace(go.Scatter(x=df_agrupado2.index, y=df_agrupado2["Muertos"]/df_agrupado2["Muertos"].shift(),
                    mode='lines',
                    name='Factor de crecimiento muertes'))
fig.update_layout(title="Factor de crecimiento casos confirmados, recuperados y muertes",
                 xaxis_title="Fecha",yaxis_title="Factor de crecimiento",
                 legend=dict(x=0,y=-0.4,traceorder="normal"))
fig.show()

Factor de crecimiento promedio casos confirmados:  1.1632748010912053
Factor de crecimiento promedio casos recuperados:  1.1591936146141768
Factor de crecimiento promedio muertes:  inf


### Estimación del modelo de predicción

In [ ]:
train_ml=df_agrupado2.iloc[:int(df_agrupado2.shape[0]*0.95)]
valid_ml=df_agrupado2.iloc[int(df_agrupado2.shape[0]*0.95):]
model_scores=[]

In [ ]:
lin_reg=LinearRegression(normalize=True)

In [ ]:
lin_reg.fit(np.array(train_ml["Días"]).reshape(-1,1),np.array(train_ml["Muertos"]).reshape(-1,1))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [ ]:
prediction_valid_linreg=lin_reg.predict(np.array(valid_ml["Días"]).reshape(-1,1))

In [177]:
model_scores.append(np.sqrt(mean_squared_error(valid_ml["Muertos"],prediction_valid_linreg)))
print("RMSE Regresión Lineal: ",np.sqrt(mean_squared_error(valid_ml["Muertos"],prediction_valid_linreg)))

RMSE Regresión Lineal:  5320.965582488232


In [178]:
plt.figure(figsize=(11,6))
prediction_linreg=lin_reg.predict(np.array(df_agrupado2["Días"]).reshape(-1,1))
linreg_output=[]
for i in range(prediction_linreg.shape[0]):
    linreg_output.append(prediction_linreg[i][0])

fig=go.Figure()
fig.add_trace(go.Scatter(x=df_agrupado2.index, y=df_agrupado2["Muertos"],
                    mode='lines+markers',name="Datos de entrenamiento"))
fig.add_trace(go.Scatter(x=df_agrupado2.index, y=linreg_output,
                    mode='lines',name="Regresión Lineal de mejor ajuste",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Muertes predicción regresión Lineal",
                 xaxis_title="Fecha",yaxis_title="Muertes",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

<Figure size 792x432 with 0 Axes>

In [ ]:
#polinomio modelo casos confirmados 
train_ml=df_agrupado2.iloc[:int(df_agrupado2.shape[0]*0.95)]
valid_ml=df_agrupado2.iloc[int(df_agrupado2.shape[0]*0.95):]

In [ ]:
poly = PolynomialFeatures(degree = 7) 

In [ ]:
train_poly=poly.fit_transform(np.array(train_ml["Días"]).reshape(-1,1))
valid_poly=poly.fit_transform(np.array(valid_ml["Días"]).reshape(-1,1))
y=train_ml["Confirmados"]

In [ ]:
linreg=LinearRegression(normalize=True)
linreg.fit(train_poly,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [ ]:
prediction_poly=linreg.predict(valid_poly)
rmse_poly=np.sqrt(mean_squared_error(valid_ml["Confirmados"],prediction_poly))
model_scores.append(rmse_poly)
print("Root Mean Squared Error for Polynomial Regression: ",rmse_poly)

Root Mean Squared Error for Polynomial Regression:  1577.2962420077342


In [ ]:
new_prediction_poly=[]
for i in range(1,173):
    new_date_poly=poly.fit_transform(np.array(df_agrupado2["Días"].max()+i).reshape(-1,1))
    new_prediction_poly.append(linreg.predict(new_date_poly)[0])
new_prediction_poly

In [184]:
comp_data=poly.fit_transform(np.array(df_agrupado2["Días"]).reshape(-1,1))
plt.figure(figsize=(11,6))
predictions_poly=linreg.predict(comp_data)

fig=go.Figure()
fig.add_trace(go.Scatter(x=df_agrupado2.index, y=df_agrupado2["Confirmados"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=df_agrupado2.index, y=predictions_poly,
                    mode='lines',name="Polynomial Regression Best Fit",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Confirmed Cases Polynomial Regression Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",
                 legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

<Figure size 792x432 with 0 Axes>

In [ ]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
model_predictions=pd.DataFrame(new_prediction_poly,
                               columns=["Dates","Polynonmial Regression Prediction"])
model_predictions.head()

In [ ]:
model_train=df_agrupado2.iloc[:int(df_agrupado2.shape[0]*0.95)]
valid=df_agrupado2.iloc[int(df_agrupado2.shape[0]*0.95):]
y_pred=valid.copy()

In [ ]:
#Ajuste de los hiperparámetros
auto_arima(model_train["Confirmados"]).get_params()

{'callback': None,
 'disp': 0,
 'maxiter': 50,
 'method': None,
 'order': (2, 1, 2),
 'out_of_sample_size': 0,
 'scoring': 'mse',
 'scoring_args': {},
 'seasonal_order': (0, 0, 0, 1),
 'solver': 'lbfgs',
 'start_params': None,
 'suppress_warnings': False,
 'transparams': True,
 'trend': 'c'}

In [ ]:
model_ar= auto_arima(model_train["Confirmados"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=10,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar.fit(model_train["Confirmados"])

Fit ARIMA: order=(0, 1, 0); AIC=2552.864, BIC=2559.052, Fit time=0.008 seconds
Fit ARIMA: order=(1, 1, 0); AIC=2543.507, BIC=2552.788, Fit time=0.051 seconds
Fit ARIMA: order=(2, 1, 0); AIC=2509.489, BIC=2521.864, Fit time=0.071 seconds
Fit ARIMA: order=(3, 1, 0); AIC=2501.826, BIC=2517.295, Fit time=0.132 seconds
Fit ARIMA: order=(4, 1, 0); AIC=2485.474, BIC=2504.036, Fit time=0.160 seconds
Fit ARIMA: order=(5, 1, 0); AIC=2484.401, BIC=2506.057, Fit time=0.287 seconds
Fit ARIMA: order=(6, 1, 0); AIC=2485.913, BIC=2510.663, Fit time=0.462 seconds
Fit ARIMA: order=(7, 1, 0); AIC=2484.211, BIC=2512.055, Fit time=1.054 seconds
Fit ARIMA: order=(8, 1, 0); AIC=2486.164, BIC=2517.101, Fit time=1.517 seconds
Fit ARIMA: order=(9, 1, 0); AIC=2484.758, BIC=2518.789, Fit time=2.328 seconds
Fit ARIMA: order=(10, 1, 0); AIC=2473.486, BIC=2510.611, Fit time=5.317 seconds
Total fit time: 11.398 seconds


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(10, 1, 0),
      out_of_sample_size=0, scoring='mse', scoring_args={}, seasonal_order=None,
      solver='lbfgs', start_params=None, suppress_warnings=True,
      transparams=True, trend='c')

In [ ]:
prediction_ar=model_ar.predict(len(valid))
y_pred["AR Model Prediction"]=prediction_ar

In [ ]:
model_scores.append(np.sqrt(mean_squared_error(y_pred["Confirmados"],y_pred["AR Model Prediction"])))
print("Root Mean Square Error for AR Model: ",np.sqrt(mean_squared_error(y_pred["Confirmados"],y_pred["AR Model Prediction"])))

Root Mean Square Error for AR Model:  2088.7076254930803


In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["Confirmados"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["Confirmados"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["AR Model Prediction"],
                    mode='lines+markers',name="Prediction of Confirmed Cases",))
fig.update_layout(title="Confirmed Cases AR Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [200]:
model_sarima= auto_arima(model_train["Confirmados"],trace=True, error_action='ignore', 
                         start_p=0,start_q=0,max_p=2,max_q=2,m=9,
                   suppress_warnings=True,stepwise=True,seasonal=True)
model_sarima.fit(model_train["Confirmados"])

Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 0, 1, 9); AIC=2548.156, BIC=2560.531, Fit time=0.425 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 9); AIC=2552.864, BIC=2559.052, Fit time=0.014 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 0, 0, 9); AIC=2539.691, BIC=2552.066, Fit time=0.304 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 1, 9); AIC=2494.346, BIC=2506.721, Fit time=0.463 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 1, 9); AIC=2490.048, BIC=2505.516, Fit time=0.911 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 0, 9); AIC=2496.840, BIC=2509.215, Fit time=0.426 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 0, 2, 9); AIC=2492.033, BIC=2510.595, Fit time=1.791 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 9); AIC=2502.239, BIC=2511.520, Fit time=0.140 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(2, 0, 2, 9); AIC=2494.607, BIC=2516.263, Fit time=2.518 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_o

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(2, 1, 2),
      out_of_sample_size=0, scoring='mse', scoring_args={},
      seasonal_order=(1, 0, 1, 9), solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend='c')

In [201]:
prediction_sarima=model_sarima.predict(len(valid))
y_pred["SARIMA Model Prediction"]=prediction_sarima


In [202]:
model_scores.append(np.sqrt(mean_squared_error(y_pred["Confirmados"],y_pred["SARIMA Model Prediction"])))
print("Root Mean Square Error for SARIMA Model: ",np.sqrt(mean_squared_error(y_pred["Confirmados"],y_pred["SARIMA Model Prediction"])))

Root Mean Square Error for SARIMA Model:  2317.878805335809


In [203]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["Confirmados"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["Confirmados"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["SARIMA Model Prediction"],
                    mode='lines+markers',name="Prediction for Confirmed Cases",))
fig.update_layout(title="Confirmed Cases SARIMA Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
SARIMA_model_new_prediction=[]
for i in range(1,18):
    SARIMA_model_new_prediction.append(model_sarima.predict(len(valid)+i)[-1])
model_predictions["SARIMA Model Prediction"]=SARIMA_model_new_prediction
model_predictions.head()